# Understand Agentic architecture concept with AutoGen Studio

----
AutoGen Studio offers four main interfaces to help you build and manage multi-agent systems:

1. Team Builder
* A visual interface for creating agent teams through declarative specification (JSON) or drag-and-drop
* Supports configuration of all core components: teams, agents, tools, models, and termination conditions
* Fully compatible with AgentChat’s component definitions
2. Playground
* Interactive environment for testing and running agent teams
* Features include:
    * Live message streaming between agents
    * Visual representation of message flow through a control transition graph
    * Interactive sessions with teams using UserProxyAgent
    * Full run control with the ability to pause or stop execution
3. Gallery
* Central hub for discovering and importing community-created components
* Enables easy integration of third-party components
4. Deployment
* Export and run teams in python code
* Setup and test endpoints based on a team configuration
* Run teams in a docker container

**Reference**
- [AutoGen Studio repo](https://github.com/microsoft/autogen/tree/main/python/packages/autogen-studio) 

In [ ]:
# you can do it in the terminal as well
!autogenstudio ui --port 8081

INFO:     Loading environment from '/home/azureuser/.autogenstudio/temp_env_vars.env'
2025-02-17 04:55:30.641 | INFO     | autogenstudio.web.initialization:__init__:40 - Initializing application data folder: /home/azureuser/.autogenstudio 
INFO:     Started server process [17361]
INFO:     Waiting for application startup.
2025-02-17 04:55:30.657 | INFO     | autogenstudio.database.db_manager:initialize_database:52 - Creating database tables...
2025-02-17 04:55:30.701 | INFO     | autogenstudio.database.schema_manager:_initialize_alembic:154 - Alembic initialization complete
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-02-17 04:55:30.734 | INFO     | autogenstudio.web.app:lifespan:35 - Application startup complete. Navigate to http://127.0.0.1:8081
2025-02-17 04:58:09.969 | INFO     | autogenstudio.database.db_manager:delete:233 - Row with filters {'id': 1, 'user_id': 'guestuser@gmail.com'} not found

## 🧪 Step 1. Set up components in your gallery 
---

1. Go to the started localhost:{your port} website.
2. Click on the `Gallery` menu.
3. Click on the `+ New Gallery` button to add a new component.
4. Upload the customized sample_gallery.json file. 
5. Check the 9 components in the gallery.

## 🧪 Step 2. Single Agent with Tools
---

1. Go to the Team Builder menu and Click on the `+ New Team` button.
2. Remove the default agent and termination condition.
3. Drag and drop the `single_tool_use_agent` and `CombinationTemination` in the team box.
4. Click the save button and goes to the Playground menu.
5. Click the `+ New Session` button to start a session and input the name of session and select your team created.
6. Click the `Create` button to run the session.
7. input your query in the input box and click the `Send` button.

![Single-Agent Workflow](images/single-agent.png)

### 🚀 Example of your messages

1. What is the history of microsoft?
2. get info from https://www.sqlite.org/copyright.html
3. what is the weather in Seoul?
4. 4.11 + 5.11?

### ✅ sample response

1. What is the history of microsoft?

![screenshot1](images/screenshot1.png)

2. get info from https://www.sqlite.org/copyright.html

![screenshot2](images/screenshot2.png)

3. what is the weather in Seoul?

![screenshot3](images/screenshot3.png)

4. 4.11 + 5.11?

![screenshot4](images/screenshot4.png)



## 🧪 Step 3. Multi-Agent (Writer and Reviewer)
---

1. Go to the Team Builder menu and Click on the `+ New Team` button.
2. Remove the default agent and termination condition.
3. Drag and drop the `writer_agent`, `reviwer_agent` and `CombinationTemination` in the team box.
4. Click the save button and goes to the Playground menu.
5. Click the `+ New Session` button to start a session and input the name of session and select your team created.
6. Click the `Create` button to run the session.
7. input your query in the input box and click the `Send` button.

![Multi-Agent Workflow](images/multi-agent.png)

### 🚀 Example of your messages

1. what is the height of the eiffel tower
2. make a bedtime story for 5 years old kids
2. make a bedtime story for 10 years old kids

### ✅ sample response

1. make a bedtime story for 10 years old kids

![screenshot5](images/screenshot5.png)

### 